In [383]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import cross_val_score


In [384]:
# yelp_raw = pd.read_csv('data/yelp_labelled.txt', 
#                        delimiter= '\t', 
#                        header=None)

# df = yelp_raw.copy()
# df.head()

imdb_raw = pd.read_csv('data/imdb_labelled.txt', 
                       delimiter= '\t', 
                       header=None)

df = imdb_raw.copy()
df.head()

# amazon_raw = pd.read_csv('data/amazon_cells_labelled.txt', 
#                          delimiter= '\t', 
#                          header=None
#                         )
# df = amazon_raw.copy()
# df.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [385]:
# set column names

df.columns = ['message', 'negative']

In [386]:
df['negative'] = (df['negative'] == 0)
# Note that if you run this cell a second time everything will become false.
# So... Don't.

In [387]:
# replace punctuation 
df['message'] = df.message.str.replace(r'[^a-zA-Z\d\s:]', '')
# make lower case
df['message'] = df['message'].str.lower()

In [388]:
# split negative messages and combine into one list
negative_words = df.message[df.negative].str.cat(sep=' ').split()

positive_words = df.message[df.negative == False].str.cat(sep=' ').split()

In [389]:
# Unique Words
print('negative:', len(np.unique(negative_words)), ' positive:', len(np.unique(positive_words)))

negative: 1928  positive: 2014


In [390]:
diff = np.setdiff1d(
        ar1=negative_words, 
                      ar2=positive_words
                     )


In [ ]:
new = df.copy()

for key in diff:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    new[key] = df['message'].apply(lambda x: key in x)

In [ ]:
new = new.iloc[:, 2:]

In [ ]:
data = new.copy()
target = df['negative']

In [ ]:
data = data.applymap(lambda x: 1 if x == True else x)
data = data.applymap(lambda x: 0 if x == False else x)

In [ ]:
words_50 = data.describe().T.sort_values(by='mean', 
                                         ascending=False).head(500).index.tolist()

In [ ]:
new = df.copy()

for key in words_50:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    new[key] = new.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )

In [ ]:
data = new.iloc[:, 2:]
target = df['negative']

In [ ]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}\nAccuracy: {}%".format(
    data.shape[0],
    (target != y_pred).sum(), round((1 - (target != y_pred).sum()/data.shape[0]) * 100, 2)
))

In [ ]:
cross_val_score(bnb, data, target, cv=5)